In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [6]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header= True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.head(1)

[Row(Email=u'mstephenson@fernandez.com', Address=u'835 Frank TunnelWrightmouth, MI 82180-9605', Avatar=u'Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [9]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.4972677251
12.6556511492
39.5776680195
4.08262063295
587.951053968


In [10]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [12]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'],outputCol = 'features')

In [13]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
output.head(1)

[Row(Email=u'mstephenson@fernandez.com', Address=u'835 Frank TunnelWrightmouth, MI 82180-9605', Avatar=u'Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [16]:
final_data = output.select('features','Yearly Amount Spent')

In [17]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [18]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [20]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                342|
|   mean|  500.1344622554146|
| stddev|  80.95501630165388|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [21]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                158|
|   mean|   497.538183784453|
| stddev|  75.86365888937465|
|    min|  304.1355915788555|
|    max|  744.2218671047146|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol= 'Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

In [25]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|0.03354721880117495|
|-2.6676623602240284|
|  6.940940315342004|
|  4.286372450285512|
| -6.602137698281922|
| 0.9801933812744892|
|  -9.02168931137578|
| 18.520415931720947|
|    7.7354717474106|
|-7.7649758261487705|
| 1.6898462602133009|
| -4.376515412595438|
| -11.57193302189603|
| -3.319242278670572|
| 1.2372131773126966|
| -6.536527626209306|
| -8.971657814514117|
|-13.471561992115767|
| -4.860579304663474|
| 16.636933169928852|
+-------------------+
only showing top 20 rows



In [27]:
test_results.rootMeanSquaredError

9.835339264484753

In [28]:
test_results.r2

0.9830851342416805

In [29]:
unlabeled_data =  test_data.select('features')

In [30]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[30.8364326747734...|
|[30.9716756438877...|
|[31.0472221394875...|
|[31.1280900496166...|
|[31.2606468698795...|
|[31.4474464941278...|
|[31.6098395733896...|
|[31.6548096756927...|
|[31.7207699002873...|
|[31.7366356860502...|
|[31.7656188210424...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8293464559211...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9365486184489...|
|[31.9673209478824...|
|[32.0180740106320...|
+--------------------+
only showing top 20 rows



In [31]:
predictions=lr_model.transform(unlabeled_data)

In [32]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...| 442.0308665392645|
|[30.8364326747734...|470.16956278721364|
|[30.9716756438877...| 487.6976694415507|
|[31.0472221394875...| 388.2110267387359|
|[31.1280900496166...| 563.8548244453366|
|[31.2606468698795...| 420.3464378756769|
|[31.4474464941278...| 427.6244314065998|
|[31.6098395733896...| 426.0251337193872|
|[31.6548096756927...| 467.5279519801379|
|[31.7207699002873...| 546.5399093041717|
|[31.7366356860502...| 495.2435999953186|
|[31.7656188210424...|500.93059704820257|
|[31.8093003166791...| 548.3438323847372|
|[31.8124825597242...| 396.1295872624678|
|[31.8293464559211...| 383.9151248106623|
|[31.8745516945853...| 398.8217718724768|
|[31.8854062999117...| 399.0749307869896|
|[31.9365486184489...|440.67094688744396|
|[31.9673209478824...|450.61042054431573|
|[32.0180740106320...|341.14617757538645|
+--------------------+------------